In [1]:
import numpy as np
import pandas as pd

### Загружаем данные

http://study.mokoron.com

In [2]:
pos = pd.read_csv('positive.csv',sep=';',error_bad_lines=False, header=None)
neg = pd.read_csv('negative.csv',sep=';',error_bad_lines=False, header=None)

In [3]:
pos.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,408906692374446080,1386325927,pleease_shut_up,"@first_timee хоть я и школота, но поверь, у на...",1,0,0,0,7569,62,61,0
1,408906692693221377,1386325927,alinakirpicheva,"Да, все-таки он немного похож на него. Но мой ...",1,0,0,0,11825,59,31,2
2,408906695083954177,1386325927,EvgeshaRe,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,0,1,0,1273,26,27,0
3,408906695356973056,1386325927,ikonnikova_21,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,0,1,0,1549,19,17,0
4,408906761416867842,1386325943,JumpyAlex,@irina_dyshkant Вот что значит страшилка :D\nН...,1,0,0,0,597,16,23,1


In [4]:
neg.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,408906762813579264,1386325944,dugarchikbellko,на работе был полный пиддес :| и так каждое за...,-1,0,0,0,8064,111,94,2
1,408906818262687744,1386325957,nugemycejela,"Коллеги сидят рубятся в Urban terror, а я из-з...",-1,0,0,0,26,42,39,0
2,408906858515398656,1386325966,4post21,@elina_4post как говорят обещаного три года жд...,-1,0,0,0,718,49,249,0
3,408906914437685248,1386325980,Poliwake,"Желаю хорошего полёта и удачной посадки,я буду...",-1,0,0,0,10628,207,200,0
4,408906914723295232,1386325980,capyvixowe,"Обновил за каким-то лешим surf, теперь не рабо...",-1,0,0,0,35,17,34,0


In [5]:
tweetdata = pd.concat([neg,pos])

In [6]:
# удаляем лишние столбцы

In [7]:
tweetdata = tweetdata.drop([0,1,2,5,6,7,8,9,10,11], axis =1)

In [8]:
tweetdata.columns = ['text','score']

In [9]:
tweetdata.head()

,text,score
0,на работе был полный пиддес :| и так каждое за...,-1
1,"Коллеги сидят рубятся в Urban terror, а я из-з...",-1
2,@elina_4post как говорят обещаного три года жд...,-1
3,"Желаю хорошего полёта и удачной посадки,я буду...",-1
4,"Обновил за каким-то лешим surf, теперь не рабо...",-1


In [10]:
# сразу перемешаем данные

In [11]:
tweetdata = tweetdata.reset_index(drop=True)

In [12]:
tweetdata = tweetdata.reindex(np.random.permutation(tweetdata.index))

In [13]:
tweetdata.tail()

,text,score
89150,"Соседи, вы дебилы, хватит в такую рань стучать...",-1
176472,Сегодня всемирный день футбола)\nС праздником ...,1
76502,RT @bezrukovaTweet: Я не перестанут кушать сла...,-1
7492,"RT @escobar199889: Зачем так люди поступают, о...",-1
152466,Мой косплей разместили в Geeks Empire и назвал...,1


In [14]:
#загрузим базу стоп слов для русского

In [15]:
import nltk

In [16]:
#nltk.download('popular')

In [17]:
from nltk.corpus import stopwords

In [18]:
import re

In [19]:
# свой токенезатор
def tokenizer (text):
    text = re.sub('http\w*',' ', text)
    text = re.sub('https\w*',' ', text)
    text = re.sub('#\w*',' ', text)
    text = re.sub('@\w*',' ', text)
    text = re.sub('©\w*',' ', text)
    text = re.sub(r'\d\w*',' ', text)
    text = re.sub('[a-zA-Z0-9_]',' ', text)
    text = re.sub(r'\\',' ', text)
    text = re.sub(r'^\w*',' ', text)
    text = re.sub(r'&',' ', text)
    text = re.sub(r'$\w*',' ', text)
    text = re.sub('%',' ', text)
    ### 0.89069962234206701 - если не удалять скобки
    text = re.sub('\(\w*',' ', text)
    text = re.sub('\)\w*',' ', text)
    text = re.sub('\[\w*',' ', text)
    text = re.sub('\]\w*',' ', text)
    text = re.sub('\|\w*',' ', text)
    text = re.sub('\{\w*',' ', text)
    text = re.sub('\}\w*',' ', text)
    ### 0.71806439288180923 - если удалять скобки
    return re.split(r'[;?!,.:\s*/"=_+-]',text)

In [20]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
vctr = CountVectorizer( stop_words=stopwords.words('russian'),tokenizer=tokenizer)

In [21]:
v = vctr.fit_transform(tweetdata['text'])

In [22]:
v.shape

(226834, 168323)

In [38]:
#sorted(vctr.vocabulary_)

### TF-IDF

In [24]:
tfidf = TfidfTransformer()
data_tfidf = tfidf.fit_transform(v)

In [25]:
data_tfidf.shape

(226834, 168323)

In [26]:
data_tfidf

<226834x168323 sparse matrix of type '<class 'numpy.float64'>'
	with 1614451 stored elements in Compressed Sparse Row format>

In [27]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_tfidf, tweetdata['score'], test_size=0.3, random_state=42)

In [28]:
print(X_train.shape,X_test.shape,X_train.shape[0] + X_test.shape[0] )
print(y_train.shape,y_test.shape,y_train.shape[0] + y_test.shape[0] )

(158783, 168323) (68051, 168323) 226834
(158783,) (68051,) 226834


### Классификация и оценка качества

In [29]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=42)

In [30]:
%%time
clf.fit(X_train,  y_train)

CPU times: user 5min 19s, sys: 296 ms, total: 5min 19s
Wall time: 5min 19s


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=42, splitter='best')

In [31]:
predicted = clf.predict(X_test)

In [32]:
from sklearn.metrics import accuracy_score, confusion_matrix

In [33]:
accuracy_score(y_test, predicted)

0.65762442873727056

In [34]:
confusion_matrix(y_test, predicted)

array([[20446, 13095],
       [10204, 24306]])

In [35]:
%%time
from sklearn import naive_bayes
clf = naive_bayes.MultinomialNB()
clf.fit(X_train, y_train)
predicted=clf.predict(X_test)

CPU times: user 44.7 ms, sys: 9.49 ms, total: 54.2 ms
Wall time: 53.6 ms


In [36]:
accuracy_score(y_test, predicted)

0.71806439288180923

In [37]:
confusion_matrix(y_test, predicted)

array([[23768,  9773],
       [ 9413, 25097]])

In [ ]:
#результаты:
# 0.74541152958810308 - без токенизатора
# 0.71806439288180923 - с токенезатором с удалением скобок
# 0.89069962234206701 - с токенезатором пропускающим скобки
